In [18]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from movies.models import Movie
from django.conf  import settings
import pandas as pd

links_path =settings.DATA_DIR / 'links.csv'
ratings_path = settings.DATA_DIR / "ratings_small.csv" 
df = pd.read_csv(ratings_path)

In [19]:
from ratings.models import Rating
qs = Rating.objects.all()
missing_movies_ids = []
for item in qs:
    if item.content_object is None:
        missing_movies_ids.append(item.object_id)

total = len(missing_movies_ids)   

In [20]:
unique_total_missing = list(set(missing_movies_ids))
print(len(unique_total_missing), total)

362 1847


In [21]:
links_df = pd.read_csv(links_path)
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [22]:

missing_df = links_df.copy([links_df.movieId.isin(unique_total_missing)])
missing_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [23]:
def enrich_col(val):
    val = str(val)
    if len(val) == 7:
        val =f"tt{val}"
        return val
    elif len(val) == 6:
        val =f"tt0{val}"
        return val
    elif len(val) == 5:
        val =f"tt00{val}"
        return val
    return val
        

In [24]:
missing_df['tt'] = missing_df['imdbId'].apply(enrich_col)


In [25]:
missing_df.head()

,movieId,imdbId,tmdbId,tt
0,1,114709,862.0,tt0114709
1,2,113497,8844.0,tt0113497
2,3,113228,15602.0,tt0113228
3,4,114885,31357.0,tt0114885
4,5,113041,11862.0,tt0113041


In [26]:
movies_path = settings.DATA_DIR / 'movies_metadata.csv'
movies_df = pd.read_csv(movies_path, usecols=["title", "overview", "release_date", "imdb_id"])
movies_df.head()


,imdb_id,overview,release_date,title
0,tt0114709,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,Toy Story
1,tt0113497,When siblings Judy and Peter discover an encha...,1995-12-15,Jumanji
2,tt0113228,A family wedding reignites the ancient feud be...,1995-12-22,Grumpier Old Men
3,tt0114885,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,Waiting to Exhale
4,tt0113041,Just when George Banks has recovered from his ...,1995-02-10,Father of the Bride Part II


In [27]:
missing_movies_df = missing_df.merge(movies_df, left_on = "tt", right_on ="imdb_id")
missing_movies_df.head()

,movieId,imdbId,tmdbId,tt,imdb_id,overview,release_date,title
0,1,114709,862.0,tt0114709,tt0114709,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,Toy Story
1,2,113497,8844.0,tt0113497,tt0113497,When siblings Judy and Peter discover an encha...,1995-12-15,Jumanji
2,3,113228,15602.0,tt0113228,tt0113228,A family wedding reignites the ancient feud be...,1995-12-22,Grumpier Old Men
3,4,114885,31357.0,tt0114885,tt0114885,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,Waiting to Exhale
4,5,113041,11862.0,tt0113041,tt0113041,Just when George Banks has recovered from his ...,1995-02-10,Father of the Bride Part II


In [28]:
missing_movies_df["id"] = missing_movies_df['movieId']
missing_movies_df["id_alt"] = missing_movies_df['tmdbId'].astype(int)
missing_movies_df.head()

,movieId,imdbId,tmdbId,tt,imdb_id,overview,release_date,title,id,id_alt
0,1,114709,862.0,tt0114709,tt0114709,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,Toy Story,1,862
1,2,113497,8844.0,tt0113497,tt0113497,When siblings Judy and Peter discover an encha...,1995-12-15,Jumanji,2,8844
2,3,113228,15602.0,tt0113228,tt0113228,A family wedding reignites the ancient feud be...,1995-12-22,Grumpier Old Men,3,15602
3,4,114885,31357.0,tt0114885,tt0114885,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,Waiting to Exhale,4,31357
4,5,113041,11862.0,tt0113041,tt0113041,Just when George Banks has recovered from his ...,1995-02-10,Father of the Bride Part II,5,11862


In [29]:
corrected_df = missing_movies_df.filter(['id', 'id_alt', 'title'])
corrected_df.head()

,id,id_alt,title
0,1,862,Toy Story
1,2,8844,Jumanji
2,3,15602,Grumpier Old Men
3,4,31357,Waiting to Exhale
4,5,11862,Father of the Bride Part II


In [30]:
alt_id_list = corrected_df['id_alt'].to_list()

In [31]:
chunk_size = 100
movies_qs = Movie.objects.all()
alt_id_chunks = [alt_id_list[i:i+chunk_size] for i in range(0, len(alt_id_list), chunk_size)]
for chunk in alt_id_chunks:
    movies_qs |= Movie.objects.filter(id__in=chunk)
movies_qs.count()

39327

In [32]:
from django.forms.models import model_to_dict

In [16]:
from django.db import IntegrityError

for obj in movies_qs:
    print(model_to_dict(obj)['id'])
    data = corrected_df.copy()[corrected_df['id_alt']==obj.id]
    if data.shape[0] == 1:
        orginal_model_data = model_to_dict(obj)
        update_data = data.to_dict('records')[0]   
        if obj.title == update_data.get('title'):
            print(update_data)
            orginal_model_data['id'] = update_data['id']
            new_model_data = {**orginal_model_data}
            print(new_model_data['id'])
            obj.delete()
            try:
                Movie.objects.create(**new_model_data)
            except IntegrityError:
                pass

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
22
23
24
25
26
27
28
29
30
31
32
34
35
36
38
39
40
41
42
43
44
45
46
47
48
49
50
52
53
54
57
58
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
78
79
80
81
82
85
86
87
88
89
90
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
110
112
113
115
116
117
118
120
121
122
123
124
125
126
127
130
132
133
135
137
138
140
141
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
168
169
170
171
172
173
174
175
176
177
178
179
180
181
183
184
185
186
187
188
189
190
191
192
193
194
195
196
198
199
201
202
203
204
205
207
208
209
211
212
213
214
216
217
218
219
220
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
246
247
248
249
250
251
252
253
255
256
257
258
259
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
285
286
289
290
291
292
294
295
296
297
298
299
300
301
302
303
305
310
312
313
314
315
316
317
319
321
322
324
32

4352
4353
4354
4355
4356
4357
4358
4359
4360
4361
4362
4363
4365
4366
4367
4368
4369
4370
4371
4372
4373
4374
4375
4376
4378
4379
4380
4381
4382
4383
4384
4385
4386
4387
4388
4389
4390
4391
4392
4393
4394
4395
4396
4397
4398
4399
4400
4401
4402
4403
4405
4406
4407
4408
4409
4410
4411
4412
4413
4414
4415
4416
4417
4418
4419
4420
4421
4422
4423
4424
4425
4426
4427
4428
4429
4430
4431
4432
4435
4436
4437
4438
4439
4440
4441
4442
4443
4444
4446
4447
4448
4450
4451
4452
4453
4454
4456
4458
4459
4462
4463
4464
4465
4466
4467
4468
4469
4470
4471
4472
4473
4474
4475
4476
4477
4478
4479
4482
4483
4485
4486
4487
4488
4489
4490
4491
4492
4493
4494
4495
4496
4497
4498
4499
4500
4501
4502
4503
4504
4505
4506
4507
4508
4509
4510
4511
4512
4514
4515
4516
4517
4518
4519
4520
4521
4522
4523
4524
4525
4526
4527
4528
4530
4531
4532
4533
4534
4535
4536
4537
4539
4540
4541
4542
4543
4544
4545
4546
4547
4548
4549
4550
4552
4553
4554
4555
4556
4557
4558
4559
4560
4561
4562
4563
4564
4565
4566
4567
4569
4570


8796
8797
8798
8799
8800
8801
8802
8803
8804
8805
8806
8807
8808
8809
8811
8812
8813
8814
8815
8816
8817
8819
8820
8821
8823
8824
8825
8826
8827
8828
8829
8830
8832
8833
8834
8835
8836
8837
8838
8839
8840
8841
8844
8846
8847
8848
8849
8851
8852
8854
8855
8857
8859
8860
8861
8862
8864
8865
8866
8868
8869
8870
8871
8872
8877
8878
8880
8881
8882
8883
8884
8885
8889
8890
8892
8893
8894
8895
8896
8897
8898
8899
8900
8901
8902
8903
8904
8907
8908
8909
8910
8911
8913
8914
8915
8917
8918
8919
8922
8923
8924
8926
8927
8928
8930
8931
8932
8933
8934
8935
8936
8937
8939
8941
8943
8946
8947
8949
8950
8951
8952
8955
8956
8957
8958
8959
8961
8962
8963
8964
8966
8967
8968
8969
8971
8972
8974
8975
8976
8977
8978
8980
8982
8983
8985
8986
8987
8988
8989
8990
8991
8992
8993
8995
8996
8997
8998
9001
9002
9003
9004
9005
9007
9008
9009
9011
9012
9013
9014
9016
9017
9018
9273
9775
10991
11115
11517
12600
13020
13209
13707
14788
15028
15257
16612
16624
18440
18837
19666
20105
20583
20650
22488
22649
22892
2294

56336
56367
56370
56389
56432
56449
56456
56479
56485
56490
56508
56511
56512
56513
56515
56516
56561
56563
56574
56580
56587
56602
56607
56612
56614
56620
56631
56633
56671
56700
56719
56748
56757
56775
56779
56782
56786
56788
56797
56801
56805
56835
56837
56846
56848
56869
56885
56888
56908
56915
56941
56945
56995
57027
57038
57041
57044
57069
57147
57171
57179
57183
57223
57236
57274
57282
57291
57359
57368
57370
57378
57401
57418
57421
57426
57432
57439
57453
57464
57468
57473
57478
57484
57499
57502
57504
57509
57522
57526
57528
57530
57532
57538
57543
57550
57637
57640
57648
57655
57669
57690
57692
57754
57772
57776
57792
57805
57843
57848
57865
57873
57910
57940
57942
57949
57951
57960
57964
57970
57972
57980
58025
58029
58033
58047
58059
58078
58103
58105
58107
58111
58123
58136
58146
58154
58156
58162
58174
58185
58223
58228
58246
58274
58282
58295
58297
58299
58303
58315
58324
58332
58334
58351
58367
58373
58379
58381
58409
58418
58422
58425
58439
58452
58485
58490
58498
5852

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

{'id': 162728, 'id_alt': 407518, 'title': 'Chosen'}
162728
407531
{'id': 175487, 'id_alt': 407531, 'title': 'Bedeviled'}
175487
407559
{'id': 165601, 'id_alt': 407559, 'title': 'I Am the Pretty Thing That Lives in the House'}
165601
407575
{'id': 173185, 'id_alt': 407575, 'title': 'Rosa Chumbe'}
173185
407588
{'id': 168982, 'id_alt': 407588, 'title': 'Floyd Norman: An Animated Life'}
168982
407613
{'id': 162412, 'id_alt': 407613, 'title': 'Brian Posehn: Criminally Posehn'}
162412
407655
{'id': 164869, 'id_alt': 407655, 'title': 'A Cinderella Story: If the Shoe Fits'}
164869
407757
{'id': 170919, 'id_alt': 407757, 'title': 'Sweet, Sweet Lonely Girl'}
170919
407789
{'id': 161924, 'id_alt': 407789, 'title': 'Clinton Cash'}
161924
407799
{'id': 165571, 'id_alt': 407799, 'title': 'The Genius of Marie Curie: The Woman Who Lit up the World'}
165571
407803
{'id': 175801, 'id_alt': 407803, 'title': 'Harry Benson: Shoot First'}
175801
407806
{'id': 164917, 'id_alt': 407806, 'title': '13th'}
1649

{'id': 164691, 'id_alt': 414771, 'title': 'The Last Band in Lebanon'}
164691
414779
{'id': 168548, 'id_alt': 414779, 'title': 'The River Thief'}
168548
414827
{'id': 163701, 'id_alt': 414827, 'title': 'Siccin 3: Cürmü Ask'}
163701
414910
{'id': 169492, 'id_alt': 414910, 'title': "Tonight It's You"}
169492
414937
{'id': 168244, 'id_alt': 414937, 'title': "Maigret's Dead Man"}
168244
414977
{'id': 166587, 'id_alt': 414977, 'title': 'Sam'}
166587
415032
{'id': 169274, 'id_alt': 415032, 'title': 'The Fabulous Patars'}
169274
415072
{'id': 3170, 'id_alt': 415072, 'title': 'Hi-Yo Silver'}
3170
415078
{'id': 165351, 'id_alt': 415078, 'title': "Hamilton's America"}
165351
415086
{'id': 175165, 'id_alt': 415086, 'title': 'What the Health'}
175165
415132
{'id': 172641, 'id_alt': 415132, 'title': 'Ares'}
172641
415214
{'id': 165581, 'id_alt': 415214, 'title': 'Operation Mekong'}
165581
415255
{'id': 170381, 'id_alt': 415255, 'title': 'The Secret of Evil'}
170381
415358
{'id': 168962, 'id_alt': 41

{'id': 169668, 'id_alt': 427451, 'title': 'Banking on Bitcoin'}
169668
427673
{'id': 174801, 'id_alt': 427673, 'title': "Sadie's Last Days on Earth"}
174801
427680
{'id': 169014, 'id_alt': 427680, 'title': 'Red Wine in the Dark Night'}
169014
428074
{'id': 168910, 'id_alt': 428074, 'title': 'Come Together'}
168910
428355
{'id': 169382, 'id_alt': 428355, 'title': 'Nick Thune: Good Guy'}
169382
428356
{'id': 170807, 'id_alt': 428356, 'title': 'Dance to Death'}
170807
428398
{'id': 171481, 'id_alt': 428398, 'title': 'Santa Claus. Battle of Mages'}
171481
428449
{'id': 168610, 'id_alt': 428449, 'title': 'A Ghost Story'}
168610
428493
{'id': 169218, 'id_alt': 428493, 'title': "God's Own Country"}
169218
428495
{'id': 175579, 'id_alt': 428495, 'title': 'The Nile Hilton Incident'}
175579
428612
{'id': 174481, 'id_alt': 428612, 'title': 'Strong Island'}
174481
428645
{'id': 168204, 'id_alt': 428645, 'title': 'Eu Fico Loko'}
168204
428687
{'id': 170447, 'id_alt': 428687, 'title': 'Deidra & Lane

{'id': 174755, 'id_alt': 445840, 'title': 'Selfie'}
174755
445916
{'id': 171801, 'id_alt': 445916, 'title': 'Asylum of Darkness'}
171801
445993
{'id': 170985, 'id_alt': 445993, 'title': 'The Marine 5: Battleground'}
170985
446048
{'id': 170201, 'id_alt': 446048, 'title': 'Jim Norton: Mouthful of Shame'}
170201
446076
{'id': 170643, 'id_alt': 446076, 'title': 'The Devotion of Suspect X'}
170643
446345
{'id': 174725, 'id_alt': 446345, 'title': "La révolution n'est pas un dîner de gala"}
174725
446493
{'id': 172051, 'id_alt': 446493, 'title': 'Moonlight in Vermont'}
172051
446850
{'id': 173481, 'id_alt': 446850, 'title': 'Three heroes and the King of the Sea'}
173481
447061
{'id': 173283, 'id_alt': 447061, 'title': 'Red Nose Day Actually'}
173283
447091
{'id': 172147, 'id_alt': 447091, 'title': 'Delisin'}
172147
447113
{'id': 171937, 'id_alt': 447113, 'title': 'Monster High: Electrified'}
171937
447169
{'id': 174119, 'id_alt': 447169, 'title': 'Arrival of McKinley’s Funeral Train at Canto

{'id': 174947, 'id_alt': 462026, 'title': 'Edith Walks'}
174947
462108
{'id': 175595, 'id_alt': 462108, 'title': "Chris D'Elia: Man on Fire"}
175595
463800
{'id': 174933, 'id_alt': 463800, 'title': 'Firebase'}
174933
464111
{'id': 175643, 'id_alt': 464111, 'title': 'Zygote'}
175643
464207
{'id': 175367, 'id_alt': 464207, 'title': 'The Truth Is in the Stars'}
175367
464819
{'id': 175137, 'id_alt': 464819, 'title': "The Making of 'The Terminator': A Retrospective"}
175137
467731
{'id': 175837, 'id_alt': 467731, 'title': 'Tragedy in a Temporary Town'}
175837
468343
{'id': 105901, 'id_alt': 468343, 'title': 'Silja - nuorena nukkunut'}
105901
468707
{'id': 176127, 'id_alt': 468707, 'title': 'Thick Lashes of Lauri Mäntyvaara'}
176127
469172
{'id': 99673, 'id_alt': 469172, 'title': 'Manuel on the Island of Wonders'}
99673


In [33]:
from ratings.tasks import task_update_movie_ratings
task_update_movie_ratings()


